# Morpheus : MVP

### Step 1: Install required python libraries

In [ ]:
! pip install mediapipe matplotlib numpy opencv-python

### Step 2: Download required models

In [ ]:
! wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

### Step 3: Imports modules

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import mediapipe as mp
from mediapipe import solutions
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

from os import listdir
from PIL import Image

### Step 4: Helpers and Utils

In [ ]:
# Just a shortcut
DrawingSpec = solutions.drawing_utils.DrawingSpec

# Folder where captured images will be saved
SAVE_DIR = './images/'

# Define number of pixels on the saved images
SAVED_IMAGE_PIXELS = 256

# Define indices of landmarks we should pay attention
# NOTE: Sorted: [superior_mouth, inferior_mouth, chin, right_ear, left_ear, right_jaw, left_jaw]
REQUIRED_LANDMARKS = [0, 17, 152, 234, 454, 172, 397]

In [ ]:
# Helper function which zoom  and scale image
def zoom(img, zoom_factor=2):
    return cv2.resize(img, None, fx=zoom_factor, fy=zoom_factor)

In [ ]:
# Helper function that plot each landmarks detected
def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

### Step 5: Core function which draws landmarks

In [ ]:
def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    # Draw landmarks on the face if there are in the REQUIRED_LANDMARKS array
    for l, landmark in enumerate(face_landmarks):
      if l in REQUIRED_LANDMARKS:
        face_landmarks_proto.landmark.extend([landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z)])

    # Just a quick demo to show its possible to measure distance between landmarks
    x1, y1, z1 = face_landmarks[0].x, face_landmarks[0].y, face_landmarks[0].z
    x2, y2, z2 = face_landmarks[17].x, face_landmarks[17].y, face_landmarks[17].z
    obm = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)
    # print(obm)

    # Define a drawing style for the landmarks
    landmarks_drawing_spec = (DrawingSpec(color=(255, 0, 0), thickness=5, circle_radius=2))

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=None,
        landmark_drawing_spec=landmarks_drawing_spec,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=None,
        landmark_drawing_spec=landmarks_drawing_spec,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=None,
          landmark_drawing_spec=landmarks_drawing_spec,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

### Step 6: Create an FaceLandmarker object.

In [ ]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

### STEP 7: Load the input image (webcam) and draw landmarks

In [ ]:
cap = cv2.VideoCapture(1) # 1 may be replaced to 0 if OS is not macos

while True:
  # Reads webcam image 
  success, img = cap.read()

  # Convert it to mediapipe format
  image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)

  # Detect face landmarks from the input image.
  detection_result = detector.detect(image)

  # Process the detection result. In this case, visualize it.
  annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

  # Flip image on vertical axis for more natural effect
  annotated_image = cv2.flip(annotated_image, 1)

  # Shows transformed image on a new window
  cv2.imshow("Image", annotated_image)

  # Check if a key is pressed on keyboard
  keypress = cv2.waitKey(1)

  # if key is s (save)
  if keypress == ord('s'):
    # Save current image in user defined folder:
    save_image = Image.fromarray(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
    resized_image = save_image.resize((SAVED_IMAGE_PIXELS, SAVED_IMAGE_PIXELS), Image.LANCZOS)
    resized_image.save(f"{SAVE_DIR}/img_{len(listdir(SAVE_DIR))}.png")

  # if key is q (quit)
  elif keypress == ord('q'):
    # Destroy all windows and release input, then quit loop
    cv2.destroyAllWindows()
    cap.release()
    break

### Step 8: Plot face blendshapes bar graph

In [ ]:
plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])

### BONUS: Test on static Image (not Video input)

In [7]:
IMAGE_PATH = 'face.png'

img = cv2.imread(IMAGE_PATH) 
image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
detection_result = detector.detect(image)
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
while True:
  cv2.imshow("Image", annotated_image)
  cv2.waitKey(0) # waitKey(0) wait until user input (keypress)